1. Решить проблему с GCV
2. Почему подтянулось на одно значение меньше
3. Пофиксить нули в изначальных данных?
4. Добавить данные через request
5. Залить на github

In [1]:
import pandas as pd

In [2]:
WH_flow_df = pd.read_csv('C:/Users/user/Documents/IDE/data/NG/Wilhelmshaven_flow_2023_05_12_160150.csv')
WH_GCV_df = pd.read_csv('C:/Users/user/Documents/IDE/data/NG/Wilhelmshaven_GCV__2023_05_12_160436.csv')
BEG_flow_df = pd.read_csv('C:/Users/user/Documents/IDE/data/NG/Baltic_Energy_Gate_flow_2023_05_12_160908.csv')
BEG_GCV_df = pd.read_csv('C:/Users/user/Documents/IDE/data/NG/Baltic_Energy_Gate_GCV__2023_05_12_160946.csv')
BBH_flow_df = pd.read_csv('C:/Users/user/Documents/IDE/data/NG/BRUNSBUETTEL_HAFEN_flow_2023_05_12_162127.csv')
BBH_GCV_df = pd.read_csv('C:/Users/user/Documents/IDE/data/NG/BRUNSBUETTEL_HAFEN_GCV_2023_05_12_161110.csv')


In [3]:
# I will use "periodFrom" as date series "index".
# Converting columns to datetime format.
WH_flow_df['periodFrom'] = pd.to_datetime(WH_flow_df['periodFrom'],dayfirst=False)
WH_GCV_df['periodFrom'] = pd.to_datetime(WH_GCV_df['periodFrom'],dayfirst=False)
BEG_flow_df['periodFrom'] = pd.to_datetime(BEG_flow_df['periodFrom'],dayfirst=False)
BEG_GCV_df['periodFrom'] = pd.to_datetime(BEG_GCV_df['periodFrom'],dayfirst=False)
BBH_flow_df['periodFrom'] = pd.to_datetime(BBH_flow_df['periodFrom'],dayfirst=False)
BBH_GCV_df['periodFrom'] = pd.to_datetime(BBH_GCV_df['periodFrom'],dayfirst=False)

In [4]:
display(BBH_flow_df.info())
display(BBH_flow_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 35 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   id                             100 non-null    object        
 1   dataSet                        100 non-null    int64         
 2   indicator                      100 non-null    object        
 3   periodType                     100 non-null    object        
 4   periodFrom                     100 non-null    datetime64[ns]
 5   periodTo                       100 non-null    object        
 6   operatorKey                    100 non-null    object        
 7   tsoEicCode                     100 non-null    object        
 8   operatorLabel                  100 non-null    object        
 9   pointKey                       100 non-null    object        
 10  pointLabel                     100 non-null    object        
 11  tsoItemIdentifier   

None

,id,dataSet,indicator,periodType,periodFrom,periodTo,operatorKey,tsoEicCode,operatorLabel,pointKey,...,isNA,originalPeriodFrom,isCmpRelevant,bookingPlatformKey,bookingPlatformLabel,bookingPlatformURL,interruptionCalculationRemark,pointType,idPointType,isArchived
0,1Physical Flowday2023-02-012023-02-02DE-TSO-00...,1,Physical Flow,day,2023-02-01 06:00:00,2023-02-02 06:00,DE-TSO-0005,21X-DE-D-A0A0A-K,Gasunie Deutschland,LNG-00060,...,NaN,NaN,NaN,PRISMA,NaN,https://platform.prisma-capacity.eu/,NaN,LNG Entry point,2,NaN
1,1Physical Flowday2023-02-022023-02-03DE-TSO-00...,1,Physical Flow,day,2023-02-02 06:00:00,2023-02-03 06:00,DE-TSO-0005,21X-DE-D-A0A0A-K,Gasunie Deutschland,LNG-00060,...,NaN,NaN,NaN,PRISMA,NaN,https://platform.prisma-capacity.eu/,NaN,LNG Entry point,2,NaN
2,1Physical Flowday2023-02-032023-02-04DE-TSO-00...,1,Physical Flow,day,2023-02-03 06:00:00,2023-02-04 06:00,DE-TSO-0005,21X-DE-D-A0A0A-K,Gasunie Deutschland,LNG-00060,...,NaN,NaN,NaN,PRISMA,NaN,https://platform.prisma-capacity.eu/,NaN,LNG Entry point,2,NaN
3,1Physical Flowday2023-02-042023-02-05DE-TSO-00...,1,Physical Flow,day,2023-02-04 06:00:00,2023-02-05 06:00,DE-TSO-0005,21X-DE-D-A0A0A-K,Gasunie Deutschland,LNG-00060,...,NaN,NaN,NaN,PRISMA,NaN,https://platform.prisma-capacity.eu/,NaN,LNG Entry point,2,NaN
4,1Physical Flowday2023-02-052023-02-06DE-TSO-00...,1,Physical Flow,day,2023-02-05 06:00:00,2023-02-06 06:00,DE-TSO-0005,21X-DE-D-A0A0A-K,Gasunie Deutschland,LNG-00060,...,NaN,NaN,NaN,PRISMA,NaN,https://platform.prisma-capacity.eu/,NaN,LNG Entry point,2,NaN


### Solving missing GCV data entries.

During the analysis I encountered this problem that for Baltic Energy Gate Terminal for some days with gas flow data there is now GCV data.  
I looked into it and there is no GCV data for the starting period of Terminal working.  
So, maybe there was some technical problems with data collection.  
Anyway, this problem could happen for any date.  
So I'm going resolve it this way:  
I will merge (by "left join") GCV data with gas flow data for each respective Terminal (keeping in mind that gas flow and GCV data should be downloaded for the same period).    
And then, if the is some missing GCV data I would fill it with 21 day rolling average (10 days before and 10 days after the date).
If there is no data for this 21 day period also - I would use average GCV for the whole period of observations.

In [5]:
BEG_flow_df = BEG_flow_df[['periodFrom','value']]
BEG_flow_df.rename(columns={"value": "Baltic_Energy_Gate_flow, kWh/d"},inplace=True)

BEG_GCV_df = BEG_GCV_df[['periodFrom','value']]
BEG_GCV_df.rename(columns={"value": "Baltic_Energy_Gate_GCV, kWh/Nm3"},inplace=True)

BEG_df = BEG_flow_df.merge(right=BEG_GCV_df, how='left',left_on='periodFrom',right_on='periodFrom')
#merged_df.drop(columns=['periodFrom', 'pointKey','pointLabel','indicator','unit'], inplace=True)
#merged_df.rename(columns={"value": "Wilhelmshaven_flow, kWh/d"},inplace=True)

In [6]:
test_df = WH_flow_df[:2]

In [7]:
display(test_df)

,id,dataSet,indicator,periodType,periodFrom,periodTo,operatorKey,tsoEicCode,operatorLabel,pointKey,...,isNA,originalPeriodFrom,isCmpRelevant,bookingPlatformKey,bookingPlatformLabel,bookingPlatformURL,interruptionCalculationRemark,pointType,idPointType,isArchived
0,1Physical Flowday2022-12-202022-12-21DE-TSO-00...,1,Physical Flow,day,2022-12-20 06:00:00,2022-12-21 06:00,DE-TSO-0009,21X-DE-C-A0A0A-T,Open Grid Europe,LNG-00066,...,NaN,NaN,NaN,PRISMA,NaN,https://platform.prisma-capacity.eu/,NaN,LNG Entry point,2,NaN
1,1Physical Flowday2022-12-212022-12-22DE-TSO-00...,1,Physical Flow,day,2022-12-21 06:00:00,2022-12-22 06:00,DE-TSO-0009,21X-DE-C-A0A0A-T,Open Grid Europe,LNG-00066,...,NaN,NaN,NaN,PRISMA,NaN,https://platform.prisma-capacity.eu/,NaN,LNG Entry point,2,NaN


In [8]:
display(test_df.iloc[1,4])

Timestamp('2022-12-21 06:00:00')

In [9]:
#BEG_df['mean_GCV'] = BEG_df['Baltic_Energy_Gate_GCV, kWh/Nm3'].rolling(21, center=True).mean(skipna=True)
#BEG_df['Baltic_Energy_Gate_GCV, kWh/Nm3'] = BEG_df['Baltic_Energy_Gate_GCV, kWh/Nm3'].fillna(0)

In [10]:
mean_GCV = BEG_df['Baltic_Energy_Gate_GCV, kWh/Nm3'].mean(skipna=True)

In [11]:
print(mean_GCV)

11.452800000000002


In [15]:
def GCV_fill(entry):
    if pd.isna(entry) == True:
        return mean_GCV  
    else:
        return entry

In [13]:
display(BEG_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141 entries, 0 to 140
Data columns (total 3 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   periodFrom                       141 non-null    datetime64[ns]
 1   Baltic_Energy_Gate_flow, kWh/d   141 non-null    int64         
 2   Baltic_Energy_Gate_GCV, kWh/Nm3  105 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 4.4 KB


None

In [16]:
BEG_df['Baltic_Energy_Gate_GCV, kWh/Nm3'] = BEG_df['Baltic_Energy_Gate_GCV, kWh/Nm3'].apply(GCV_fill) 

In [17]:
display(BEG_df)
display(BEG_df[BEG_df['periodFrom'] == '2023-01-15 06:00:00'])
display(BEG_df.info())

,periodFrom,"Baltic_Energy_Gate_flow, kWh/d","Baltic_Energy_Gate_GCV, kWh/Nm3"
0,2022-12-22 06:00:00,0,11.4528
1,2022-12-23 06:00:00,0,11.4528
2,2022-12-24 06:00:00,0,11.4528
3,2022-12-25 06:00:00,0,11.4528
4,2022-12-26 06:00:00,0,11.4528
...,...,...,...
136,2023-05-07 06:00:00,36134838,11.3280
137,2023-05-08 06:00:00,36100314,11.3190
138,2023-05-09 06:00:00,24147517,11.3110
139,2023-05-10 06:00:00,35871124,11.3270


,periodFrom,"Baltic_Energy_Gate_flow, kWh/d","Baltic_Energy_Gate_GCV, kWh/Nm3"
24,2023-01-15 06:00:00,18478752,11.4528


<class 'pandas.core.frame.DataFrame'>
Int64Index: 141 entries, 0 to 140
Data columns (total 3 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   periodFrom                       141 non-null    datetime64[ns]
 1   Baltic_Energy_Gate_flow, kWh/d   141 non-null    int64         
 2   Baltic_Energy_Gate_GCV, kWh/Nm3  141 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 4.4 KB


None

In [ ]:
df = pd.DataFrame({'A': [0, 1, 0, 1, 0, 1, 1, 1, 0, 0]})

In [ ]:
df['B'] = df['A'].rolling(3, center=True).mean()

In [ ]:
display(df)

In [ ]:
# Getting rid of some columns
WH_flow_df = WH_flow_df[['periodFrom','pointKey','pointLabel','indicator','value','unit']]
WH_GCV_df = WH_GCV_df[['periodFrom','pointKey','pointLabel','indicator','value','unit']]
BEG_flow_df = BEG_flow_df[['periodFrom','pointKey','pointLabel','indicator','value','unit']]
BEG_GCV_df = BEG_GCV_df[['periodFrom','pointKey','pointLabel','indicator','value','unit']]
BBH_flow_df = BBH_flow_df[['periodFrom','pointKey','pointLabel','indicator','value','unit']]
BBH_GCV_df = BBH_GCV_df[['periodFrom','pointKey','pointLabel','indicator','value','unit']]

In [ ]:
display(WH_flow_df.info())
display(WH_flow_df.head())

Now I want to create a dataframe where we'll have a date range as "index".  
Then I'll add flow and GCV data by merging ("left join") data to this "index".  
The point is that downloaded dataframe with terminal's data have different amount of entries,  
as Terminals have gas flows on different dates.  
So this "index" will serve as base for this dataframe.

In [ ]:
# Creating date range.
merged_df = pd.DataFrame(pd.date_range(start='2022-09-01 06:00:00', end='2023-05-10 06:00:00', freq='D'), columns=['date'])

In [ ]:
# Merging dataframes.
merged_df = merged_df.merge(right=WH_flow_df, how='left',left_on='date',right_on='periodFrom')
merged_df.drop(columns=['periodFrom', 'pointKey','pointLabel','indicator','unit'], inplace=True)
merged_df.rename(columns={"value": "Wilhelmshaven_flow, kWh/d"},inplace=True)

merged_df = merged_df.merge(right=WH_GCV_df, how='left',left_on='date',right_on='periodFrom')
merged_df.drop(columns=['periodFrom', 'pointKey','pointLabel','indicator','unit'], inplace=True)
merged_df.rename(columns={"value": "Wilhelmshaven_GCV, kWh/Nm3"},inplace=True)

merged_df = merged_df.merge(right=BEG_flow_df, how='left',left_on='date',right_on='periodFrom')
merged_df.drop(columns=['periodFrom', 'pointKey','pointLabel','indicator','unit'], inplace=True)
merged_df.rename(columns={"value": "Baltic_Energy_Gate_flow, kWh/d"},inplace=True)

merged_df = merged_df.merge(right=BEG_GCV_df, how='left',left_on='date',right_on='periodFrom')
merged_df.drop(columns=['periodFrom', 'pointKey','pointLabel','indicator','unit'], inplace=True)
merged_df.rename(columns={"value": "Baltic_Energy_Gate_GCV, kWh/Nm3"},inplace=True)

merged_df = merged_df.merge(right=BBH_flow_df, how='left',left_on='date',right_on='periodFrom')
merged_df.drop(columns=['periodFrom', 'pointKey','pointLabel','indicator','unit'], inplace=True)
merged_df.rename(columns={"value": "Brunsbuettel_Hafen_flow, kWh/d"},inplace=True)

merged_df = merged_df.merge(right=BBH_GCV_df, how='left',left_on='date',right_on='periodFrom')
merged_df.drop(columns=['periodFrom', 'pointKey','pointLabel','indicator','unit'], inplace=True)
merged_df.rename(columns={"value": "Brunsbuettel_Hafen_GCV, kWh/Nm3"},inplace=True)


In [ ]:
# Converting kWh/day to
merged_df['Wilhelmshaven_flow, kWh/d'] = merged_df['Wilhelmshaven_flow, kWh/d']/1000/1000
merged_df['Baltic_Energy_Gate_flow, kWh/d'] = merged_df['Baltic_Energy_Gate_flow, kWh/d']/1000/1000
merged_df['Brunsbuettel_Hafen_flow, kWh/d'] = merged_df['Brunsbuettel_Hafen_flow, kWh/d']/1000/1000

In [ ]:
# Renaming columns accordingly
merged_df.rename(columns={"Wilhelmshaven_flow, kWh/d": "Wilhelmshaven_flow, GWh/d"},inplace=True)
merged_df.rename(columns={"Baltic_Energy_Gate_flow, kWh/d": "Baltic_Energy_Gate_flow, GWh/d"},inplace=True)
merged_df.rename(columns={"Brunsbuettel_Hafen_flow, kWh/d": "Brunsbuettel_Hafen_flow, GWh/d"},inplace=True)

In [ ]:
display(merged_df.head())
display(merged_df.info())
display(merged_df[merged_df['date'] == '2023-04-01 06:00:00'])

Now I want to create visualization for these gas flows using matplotlib library.  
I want to create two stackplots for flow in GWh/day and in 1000 m3/day.

In [ ]:
display(merged_df[merged_df['Baltic_Energy_Gate_GCV, kWh/Nm3'].isnull() == True])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#fig, ax = plt.subplots()
fig = plt.figure(figsize=(8, 4))
ax = fig.add_axes([0, 0, 1, 1])
ax.stackplot(merged_df['date'], merged_df['Wilhelmshaven_flow, GWh/d'], merged_df['Brunsbuettel_Hafen_flow, GWh/d'], merged_df['Baltic_Energy_Gate_flow, GWh/d'],
             labels=['Wilhelmshaven', 'Brunsbuettel','Baltic_Energy_Gate'], alpha=0.8,
             #colors = ['mediumblue', 'darkred', 'lawngreen']
             )
ax.legend(loc='upper left')
ax.set_title('Gas Flows at German LNG Terminals by day, GWh/day')
#ax.set_xlabel('Day')
ax.set_ylabel('GWh/day')

plt.show()